![](https://www.wias-berlin.de/research/ats/imaging/overview.png)wias-berlin.de

#Code by https://docs.fast.ai/medical.imaging.html#Dataset.set_pixels

In [ ]:
!pip install fastai

In [ ]:
!pip install pydicom

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pydicom
from fastai.vision.all import *
from fastai.medical.imaging import *

try:
    import cv2
    cv2.setNumThreads(0)
except: pass

import seaborn as sns
sns.set(style="whitegrid")
sns.set_context("paper")

In [ ]:
!pip install scikit-image

In [ ]:
pip install kornia===0.2.0

In [ ]:
#Code by Amrit Virdee  https://www.kaggle.com/avirdee/rsna-miccai-initial-fmi-fastai/notebook

mri = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
train_files = get_dicom_files(f'{source}/train')
labels = pd.read_csv(f'{source}/train_labels.csv')
print(os.listdir(source))

In [ ]:
#get dicom files
items = get_dicom_files(mri, recurse=True, folders='train')
items

In [ ]:
Sample_DCM = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00688/T2w/Image-273.dcm')
dcm = Sample_DCM.dcmread()
dcm

In [ ]:
type(dcm)

In [ ]:
dcm.pixels

#None

scaled_px uses RescaleSlope and RescaleIntercept values to correctly scale the image so that they represent the correct tissue densities.

In [ ]:
plt.hist(dcm.pixels.flatten().numpy());

In [ ]:
plt.hist(dcm.scaled_px.flatten().numpy());

In [ ]:
t_bin = dcm.pixels.freqhist_bins(n_bins=1)
t_bin

#Tensor.freqhist_bins

A function to split the range of pixel values into groups, such that each group has around the same number of pixels.

In [ ]:
plt.hist(t_bin.numpy(), bins=t_bin, color='c')
plt.plot(t_bin, torch.linspace(0,1,len(t_bin)));

#with n_bins at 100

In [ ]:
t_bin = dcm.pixels.freqhist_bins(n_bins=100)
t_bin

In [ ]:
plt.hist(t_bin.numpy(), bins=t_bin, color='c'); plt.plot(t_bin, torch.linspace(0,1,len(t_bin)));

In [ ]:
plt.hist(dcm.pixels.flatten().numpy(), bins=100);

#Tensor.hist_scaled

Scales a tensor using freqhist_bins to values between 0 and 1

The test image has pixel values that range between -1000 and 2500

In [ ]:
tensor_hists = dcm.pixels.hist_scaled()
plt.hist(tensor_hists.flatten().numpy(), bins=100);

#Dataset.hist_scaled

Dataset.hist_scaled(brks=None, min_px=None, max_px=None)

Pixels scaled to a min_px and max_px value

In [ ]:
data_scaled = dcm.hist_scaled()
plt.imshow(data_scaled, cmap=plt.cm.bone);

In [ ]:
data_scaled = dcm.hist_scaled(min_px=100, max_px=1000)
plt.imshow(data_scaled, cmap=plt.cm.bone);

Dicom images can contain a high amount of voxel values and windowing can be thought of as a means of manipulating these values in order to change the apperance of the image so particular structures are highlighted. A window has 2 values:

l = window level or center aka brightness

w = window width or range aka contrast

#Dataset.windowed

Dataset.windowed(w, l)

In [ ]:
plt.imshow(dcm.windowed(*dicom_windows.brain), cmap=plt.cm.bone);

#Dataset.show

Dataset.show(frames=1, scale=True, cmap=<matplotlib.colors.LinearSegmentedColormap object at 0x7f1cb02ce6a0>, min_px=-1100, max_px=None, **kwargs)

Adds functionality to view dicom images where each file may have more than 1 frame.

In [ ]:
scales = False, True, dicom_windows.brain, dicom_windows.subdural
titles = 'raw','normalized','brain windowed','subdural windowed'
for s,a,t in zip(scales, subplots(2,2,imsize=4)[1].flat, titles):
    dcm.show(scale=s, ax=a, title=t)

In [ ]:
dcm.show(cmap=plt.cm.gist_ncar, figsize=(6,6))

In [ ]:
dcm.show()

#Dataset.pct_in_window

Dataset.pct_in_window(dcm:Dataset, w, l)

% of pixels in the window (w,l)

In [ ]:
dcm.pct_in_window(*dicom_windows.brain)

#uniform_blur2d

uniform_blur2d(x, s)

Uniformly apply blurring

In [ ]:
ims = dcm.hist_scaled(), uniform_blur2d(dcm.hist_scaled(), 20), uniform_blur2d(dcm.hist_scaled(), 50)
show_images(ims, titles=('original', 'blurred 20', 'blurred 50'))

#gauss_blur2d

gauss_blur2d(x, s)

Apply gaussian_blur2d kornia filter

In [ ]:
ims = dcm.hist_scaled(), gauss_blur2d(dcm.hist_scaled(), 20), gauss_blur2d(dcm.hist_scaled(), 50)
show_images(ims, titles=('original', 'gauss_blur 20', 'gauss_blur 50'))

#Tensor.mask_from_blur

Tensor.mask_from_blur(x:Tensor, window, sigma=0.3, thresh=0.05, remove_max=True)

Create a mask from the blurred image

#Dataset.mask_from_blur

Dataset.mask_from_blur(x:Dataset, window, sigma=0.3, thresh=0.05, remove_max=True)

Create a mask from the blurred image.

In [ ]:
mask = dcm.mask_from_blur(dicom_windows.brain, sigma=0.9, thresh=0.1, remove_max=True)
wind = dcm.windowed(*dicom_windows.brain)

_,ax = subplots(1,3)
show_image(wind, ax=ax[0], title='window')
show_image(mask, alpha=0.5, cmap=plt.cm.Reds, ax=ax[1], title='mask')
show_image(wind, ax=ax[2])
show_image(mask, alpha=0.5, cmap=plt.cm.Reds, ax=ax[2], title='window and mask');

#mask2bbox

mask2bbox(mask)

In [ ]:
bbs = mask2bbox(mask)
lo,hi = bbs
show_image(wind[lo[0]:hi[0],lo[1]:hi[1]]);

#crop_resize

crop_resize(x, crops, new_sz)

In [ ]:
px256 = crop_resize(to_device(wind[None]), bbs[...,None], 128)[0]
show_image(px256)
px256.shape

#Comparing the original image with the image from using the mask and crop_resize function

In [ ]:
_,axs = subplots(1,2)
dcm.show(ax=axs[0])
show_image(px256, ax=axs[1]);

#Tensor.to_nchan

Tensor.to_nchan(x:Tensor, wins, bins=None)

#Dataset.to_nchan

Dataset.to_nchan(x:Dataset, wins, bins=None)

to_nchan takes a tensor or a dicom as the input and returns multiple one channel images (the first depending on the choosen windows and a normalized image). Setting bins to 0 only returns the windowed image.

In [ ]:
show_images(dcm.to_nchan([dicom_windows.brain], bins=0))

In [ ]:
show_images(dcm.to_nchan([dicom_windows.brain], bins=None))

#Tensor.to_3chan

Tensor.to_3chan(x:Tensor, win1, win2, bins=None)

#Dataset.to_3chan

Dataset.to_3chan(x:Dataset, win1, win2, bins=None)

In [ ]:
show_images(dcm.to_nchan([dicom_windows.brain,dicom_windows.subdural,dicom_windows.abdomen_soft]))

#Dataset.save_tif16

Dataset.save_tif16(x:Dataset'>), path, bins=None, compress=True)

Save tensor or dicom image into tiff format.

In [ ]:
_,axs=subplots(1,2)
with tempfile.TemporaryDirectory() as f:
    f = Path(f)
    dcm.save_jpg(f/'test.jpg', [dicom_windows.brain,dicom_windows.subdural])
    show_image(Image.open(f/'test.jpg'), ax=axs[0])
    dcm.save_tif16(f/'test.tif')
    show_image(Image.open(str(f/'test.tif')), ax=axs[1]);

#Dataset.set_pixels

In [ ]:
dcm.pixel_array.shape

#Dataset.zoom

Dataset.zoom(ratio)

Zoom image by specified ratio

Check to see the current size of the dicom image

In [ ]:
dcm.zoom(7.0)
dcm.show(); dcm.pixel_array.shape

#Dataset.zoom_to

Dataset.zoom_to(sz)

Change image size to specified pixel size

In [ ]:
dcm.zoom_to(200); dcm.pixel_array.shape

In [ ]:
dcm2 = TEST_DCM.dcmread()
dcm2.zoom_to(90)
test_eq(dcm2.shape, (90,90))

In [ ]:
dcm2 = TEST_DCM.dcmread()
dcm2.zoom(0.25)
dcm2.show()

#All script by Fastai Medical Imaging. Helpers for working with DICOM files

https://docs.fast.ai/medical.imaging.html#Dataset.set_pixels